In [16]:
import numpy as np
import sys
import os
import re
import obf_support

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

 Parse MSR file and print metadata from all confocal images of either a single msr file, or all msr files in a folder.

In [22]:
folder = os.path.join(parentdir, 'exampledata\\cav1\\site-sm\\240209')
files = os.listdir(folder)
files_msr = [file for file in files if '.msr' in file]
print(files_msr)

['240209-123152_minflux.msr', '240209-155203_minflux.msr', '240209-160307_minflux.msr']


In [23]:
# get metadata from confocal images in one msr file (pixel size, image shape, image size, origin offset)
file_idx = 0
msr_dataset = obf_support.File(os.path.join(folder,files_msr[file_idx]))
for conf_msr_stack_index in range(len(msr_dataset.stacks)):
    conf_stack = msr_dataset.stacks[conf_msr_stack_index]
    pxsize = conf_stack.pixel_sizes[0]*1e6
    pxshift = np.array([pxsize/2,pxsize/2])
    conf_size_px = (conf_stack.shape[0], conf_stack.shape[1])
    conf_size = (conf_stack.lengths[0]*1e6, conf_stack.lengths[1]*1e6)
    conf_scanspeed_off = (-0.02,0)
    conf_offset = (conf_stack.offsets[0]*1e6+conf_scanspeed_off[0], conf_stack.offsets[1]*1e6+conf_scanspeed_off[1])
    info = vars(conf_stack)
    dwelltime_us = float(info['tag_dictionary']['imspector'][info['tag_dictionary']['imspector'].find('dwelltime_per_pixel idx="1" type="double" size="1 1"')+53:info['tag_dictionary']['imspector'].find('/dwelltime_per_pixel')-1])*1e6
    bidirectional_bool = info['tag_dictionary']['imspector'][info['tag_dictionary']['imspector'].find('bidirectional_scan idx="1" type="logical" size="1 1"')+53:info['tag_dictionary']['imspector'].find('/bidirectional_scan')-1]
    # get only confocal stacks
    if len(bidirectional_bool) < 10:
        print('')
        print(f'{dwelltime_us} µs')
        print(bidirectional_bool)
        idx_start = [m.start() for m in re.finditer('<len idx="1" type="double" size="1 1">', info['tag_dictionary']['imspector'])]
        idx_stop = [m.start() for m in re.finditer('</len>', info['tag_dictionary']['imspector'])]
        conf_im_sizex = float(info['tag_dictionary']['imspector'][idx_start[1]+38:idx_stop[1]])*1e6
        conf_im_sizey = float(info['tag_dictionary']['imspector'][idx_start[2]+38:idx_stop[2]])*1e6
        idx_start = [m.start() for m in re.finditer('<psz idx="1" type="double" size="1 1">', info['tag_dictionary']['imspector'])]
        idx_stop = [m.start() for m in re.finditer('</psz>', info['tag_dictionary']['imspector'])]
        pixelsize = float(info['tag_dictionary']['imspector'][idx_start[1]+38:idx_stop[1]])*1e9
        print(f'{conf_im_sizex:.1f} x {conf_im_sizey:.1f} µm2')
        print(f'{pixelsize:.1f} nm')
    else:
        #print('')
        #print('Non-confocal stack')
        pass


2.0 µs
1
80.0 x 80.0 µm2
100.0 nm

2.0 µs
1
50.0 x 40.0 µm2
50.0 nm

2.0 µs
1
50.0 x 40.0 µm2
50.0 nm

2.0 µs
1
50.0 x 40.0 µm2
50.0 nm

2.0 µs
1
5.1 x 4.9 µm2
100.0 nm


In [20]:
## checking and printing confocal settings for all data in a folder structure
top_path = 'C:\\Users\\alvelidjonatan\\Documents\\Data\\etMINFLUX-lab\\data-dyn1\\memb-3d\\ves\\'
eventfolders_all = []
datefolders = os.listdir(top_path)
for datefolder in datefolders:
    if 'badevents' not in datefolder and 'test' not in datefolder and os.path.isdir(os.path.join(top_path, datefolder)):
        samplefolders = os.listdir(os.path.join(top_path, datefolder))
        for samplefolder in samplefolders:
            if os.path.isdir(os.path.join(top_path, datefolder, samplefolder)):
                cellfolders = os.listdir(os.path.join(top_path, datefolder, samplefolder))
                for cellfolder in cellfolders:
                    eventfolders = os.listdir(os.path.join(top_path, datefolder, samplefolder, cellfolder))
                    for eventfolder in eventfolders:
                        if os.path.isdir(os.path.join(top_path, datefolder, samplefolder, cellfolder, eventfolder)):
                            if 'nomfx' not in eventfolder:
                                if 'manual' not in eventfolder:
                                    if any(filename.endswith('.npy') for filename in os.listdir(os.path.join(top_path, datefolder, samplefolder, cellfolder, eventfolder))):
                                        eventfolders_all.append(os.path.join(top_path, datefolder, samplefolder, cellfolder, eventfolder))
print(eventfolders_all)

for eventfolder in eventfolders_all:
    files = os.listdir(eventfolder)
    files_msr = [file for file in files if '.msr' in file]
    print(files_msr)
    file_idx = 0
    
    # get metadata from confocal image in msr file (pixel size, image shape, image size, origin offset)
    msr_dataset = obf_support.File(os.path.join(eventfolder, files_msr[file_idx]))
    for conf_msr_stack_index in range(len(msr_dataset.stacks)):
        conf_stack = msr_dataset.stacks[conf_msr_stack_index]
        info = vars(conf_stack)
        dwelltime_us = float(info['tag_dictionary']['imspector'][info['tag_dictionary']['imspector'].find('dwelltime_per_pixel idx="1" type="double" size="1 1"')+53:info['tag_dictionary']['imspector'].find('/dwelltime_per_pixel')-1])*1e6
        bidirectional_bool = info['tag_dictionary']['imspector'][info['tag_dictionary']['imspector'].find('bidirectional_scan idx="1" type="logical" size="1 1"')+53:info['tag_dictionary']['imspector'].find('/bidirectional_scan')-1]
        # get only confocal stacks
        if len(bidirectional_bool) < 10:
            idx_start = [m.start() for m in re.finditer('<len idx="1" type="double" size="1 1">', info['tag_dictionary']['imspector'])]
            idx_stop = [m.start() for m in re.finditer('</len>', info['tag_dictionary']['imspector'])]
            # get only confocal stacks between 5.1-50 µm axis length
            if float(info['tag_dictionary']['imspector'][idx_start[1]+38:idx_stop[1]])*1e6 < 50 and float(info['tag_dictionary']['imspector'][idx_start[1]+38:idx_stop[1]])*1e6 > 5.1:
                print(f'{dwelltime_us} µs')
                print(bidirectional_bool)
                idx_start = [m.start() for m in re.finditer('<len idx="1" type="double" size="1 1">', info['tag_dictionary']['imspector'])]
                idx_stop = [m.start() for m in re.finditer('</len>', info['tag_dictionary']['imspector'])]
                conf_im_sizex = float(info['tag_dictionary']['imspector'][idx_start[1]+38:idx_stop[1]])*1e6
                conf_im_sizey = float(info['tag_dictionary']['imspector'][idx_start[2]+38:idx_stop[2]])*1e6
                idx_start = [m.start() for m in re.finditer('<psz idx="1" type="double" size="1 1">', info['tag_dictionary']['imspector'])]
                idx_stop = [m.start() for m in re.finditer('</psz>', info['tag_dictionary']['imspector'])]
                pixelsize = float(info['tag_dictionary']['imspector'][idx_start[1]+38:idx_stop[1]])*1e9
                print(f'{conf_im_sizex:.1f} x {conf_im_sizey:.1f} µm2')
                print(f'{pixelsize:.1f} nm')
                print('')
        else:
            #print('')
            #print('Non-confocal stack')
            pass

['C:\\Users\\alvelidjonatan\\Documents\\Data\\etMINFLUX-lab\\data-dyn1\\memb-3d\\ves\\241031\\sample2\\cell1\\e2', 'C:\\Users\\alvelidjonatan\\Documents\\Data\\etMINFLUX-lab\\data-dyn1\\memb-3d\\ves\\241031\\sample2\\cell2\\e2', 'C:\\Users\\alvelidjonatan\\Documents\\Data\\etMINFLUX-lab\\data-dyn1\\memb-3d\\ves\\241031\\sample2\\cell4\\e1', 'C:\\Users\\alvelidjonatan\\Documents\\Data\\etMINFLUX-lab\\data-dyn1\\memb-3d\\ves\\241031\\sample2\\cell6\\e4', 'C:\\Users\\alvelidjonatan\\Documents\\Data\\etMINFLUX-lab\\data-dyn1\\memb-3d\\ves\\241031\\sample2\\cell6\\e9', 'C:\\Users\\alvelidjonatan\\Documents\\Data\\etMINFLUX-lab\\data-dyn1\\memb-3d\\ves\\241031\\sample3\\cell2\\e1', 'C:\\Users\\alvelidjonatan\\Documents\\Data\\etMINFLUX-lab\\data-dyn1\\memb-3d\\ves\\241031\\sample3\\cell2\\e6', 'C:\\Users\\alvelidjonatan\\Documents\\Data\\etMINFLUX-lab\\data-dyn1\\memb-3d\\ves\\241031\\sample3\\cell2\\e7', 'C:\\Users\\alvelidjonatan\\Documents\\Data\\etMINFLUX-lab\\data-dyn1\\memb-3d\\ves\\24